## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-03-20-53-37 +0000,nyt,House Republicans Subpoena Jack Smith Over Pro...,https://www.nytimes.com/2025/12/03/us/politics...
1,2025-12-03-20-50-00 +0000,wsj,Immigration Sweep Begins in New Orleans as Pro...,https://www.wsj.com/us-news/new-orleans-louisi...
2,2025-12-03-20-48-59 +0000,nyt,South Korea Says the U.S. Agreed to Help It Se...,https://www.nytimes.com/2025/12/03/world/asia/...
3,2025-12-03-20-40-22 +0000,nyt,National Park Service Will Close All Grand Can...,https://www.nytimes.com/2025/12/03/travel/gran...
4,2025-12-03-20-40-00 +0000,wsj,A Pentagon watchdog has found Defense Secretar...,https://www.wsj.com/politics/national-security...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,61
10,new,19
172,ukraine,14
46,he,14
26,will,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
336,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...,181
298,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over President Trump by ...",https://www.wsj.com/politics/national-security...,134
292,2025-12-03-02-19-03 +0000,cbc,Trump says National Guard will soon go to New ...,https://www.cbc.ca/news/world/trump-new-orlean...,116
304,2025-12-03-01-41-00 +0000,wsj,President Trump on Tuesday lashed out against ...,https://www.wsj.com/politics/policy/ice-plans-...,111
306,2025-12-03-01-35-58 +0000,nypost,Trump travel ban could expand to 30 countries ...,https://nypost.com/2025/12/02/us-news/trump-tr...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
336,181,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...
298,73,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over President Trump by ...",https://www.wsj.com/politics/national-security...
334,59,2025-12-02-23-45-31 +0000,latimes,"Baby, baby, baby, baby, oh, baby: New gorilla ...",https://www.latimes.com/california/story/2025-...
173,59,2025-12-03-13-33-53 +0000,nypost,Arrogant Putin sees ‘no point in making any se...,https://nypost.com/2025/12/03/world-news/putin...
201,56,2025-12-03-11-41-00 +0000,wsj,House Republicans held on to a congressional d...,https://www.wsj.com/politics/elections/republi...
306,49,2025-12-03-01-35-58 +0000,nypost,Trump travel ban could expand to 30 countries ...,https://nypost.com/2025/12/02/us-news/trump-tr...
216,42,2025-12-03-11-00-00 +0000,nypost,Sen. Ron Johnson tells ‘Pod Force One’ he was ...,https://nypost.com/2025/12/03/us-news/sen-ron-...
92,40,2025-12-03-17-51-25 +0000,nyt,"Israel Says a Gaza Border Will Reopen, but Onl...",https://www.nytimes.com/2025/12/03/world/middl...
123,38,2025-12-03-16-39-00 +0000,wsj,EU Launches $3.5 Billion Plan to Secure Raw-Ma...,https://www.wsj.com/world/europe/eu-launches-3...
291,36,2025-12-03-02-25-12 +0000,missionlocal,‘I don’t want to come to school tomorrow:’ S.F...,https://missionlocal.org/2025/12/student-shot-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
